In [2]:
!pip install pycryptodome
!pip install filetype
!pip install sympy

     |████████████████████████████████| 6.5 MB 945 kB/s eta 0:00:01
     |████████████████████████████████| 532 kB 8.2 MB/s eta 0:00:01


In [24]:
from Crypto.Cipher import AES
import filetype
from Crypto.PublicKey import RSA
from glob import glob
from math import gcd
from sympy import *
import os

In [25]:
ruta = "./keys/"

In [26]:
aes = ruta + "pablo.vega.gallego_AES_pseudo.enc"

In [27]:
pem = ruta + "pablo.vega.gallego_pubkeyRSA_pseudo.pem"

#pem = ruta + 'natasha.trojan_pubkeyRSA_pseudo.pem'

In [28]:
rsa = ruta + "pablo.vega.gallego_RSA_pseudo.enc"

In [29]:
with open(pem, 'r') as f:
    rsakey = f.read()

pubkey = RSA.importKey(rsakey)
personal = pubkey.n
public_exponent = pubkey.e
print(personal)
print()
print(public_exponent)

17181231366303391873898577062930202499686979673305764574003306725162865918368116170678711032740944866138728394952134921252076775494646966652528085073991005136796750544201756132167645132364489116331195797880318163043267901242371677507354398880152587459039524433867736413865889982908266981823772901806382704891433640877282195304210220797188640215793024246713166930444294299041192797079131083907161337063825387274276191724839801313077693863230989409842085334969211920098270734758450805660228044269541846233922766893573054831835886332001513741764232672226913247574014565476793791378670286881130358838909791886138854955169

65537


In [30]:
len(bin(personal)[2:]) % 4 == 0

True

In [31]:
N = len(bin(personal)[2:])//4
N

512

In [32]:
binpersona = bin(personal)[2:]

In [33]:
primero = binpersona[0:N]
segundo = binpersona[N:2*N]
tercero = binpersona[2*N:3*N]
cuarto = binpersona[3*N:4*N]

In [34]:
personal == int(primero+segundo+tercero+cuarto, 2)

True

In [35]:
var('x')
for i in range(0, 2**N):
    c = int(bin(i)[2:] + segundo + tercero, 2)
    primeroAct = bin(int(primero, 2) - i)[2:]
    RS = int(primeroAct + cuarto, 2)
    RSinv = int(cuarto + primeroAct, 2)
    r2s2 = c - RSinv
    rs2 = r2s2 + 2*RS
    rs = sqrt(rs2)
    sol = solve(x**2 - x*rs + RS, x)
    if type(sol[0]) == Integer:
        print(sol)
        break

[8120470032484937215252352830373054957005429959946418105737831966927293227804430640111052886726065514448123577935787451501083134212657907371004421738067647, 11769487670416003154947777018232547194230957256023044363985701026726845357363799664299211884815687412739135955509843804465741245736272155354245040492439711]


In [36]:
sol

[8120470032484937215252352830373054957005429959946418105737831966927293227804430640111052886726065514448123577935787451501083134212657907371004421738067647,
 11769487670416003154947777018232547194230957256023044363985701026726845357363799664299211884815687412739135955509843804465741245736272155354245040492439711]

In [37]:
p = int(bin(sol[0])[2:] + bin(sol[1])[2:], 2)
q = int(bin(sol[1])[2:] + bin(sol[0])[2:], 2)

In [38]:
p * q == personal

True

In [39]:
phi = (p-1)*(q-1)

private_exponent = pow(public_exponent, -1, phi)

key = RSA.construct((int(personal), int(public_exponent), int(private_exponent)), consistency_check=False)

In [40]:
f = open('privateKey_Pablo_Vega.pem','wb')
f.write(key.exportKey('PEM'))
f.close()

In [41]:
os.system('openssl rsautl -decrypt -in pablo.vega.gallego_RSA_pseudo.enc -out AESkey.txt -inkey privateKey_Pablo_Vega.pem')

0

In [42]:
os.system('openssl enc -d -aes-128-cbc -pbkdf2 -kfile AESkey.txt -in pablo.vega.gallego_AES_pseudo.enc -out desencriptado.txt')

0

In [43]:
with open('desencriptado.txt', 'rb') as f:
    desencriptado = f.read()

In [44]:
tipo = filetype.guess(desencriptado)
extension = tipo.extension
print(extension)

jpg


In [45]:
with open('PabloVega.' + extension, "wb") as binary_file:
        binary_file.write(desencriptado)